# Exploratory Data Analysis and Data Cleaning

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
%matplotlib inline
sns.set()

In [106]:
crashes = pd.read_csv('Traffic_Crashes_-_Crashes.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'Traffic_Crashes_-_Crashes.csv'

In [3]:
# Low_memory=False, because it has mixed types of data in the same column/too many rows
people = pd.read_csv('Traffic_Crashes_-_People.csv', low_memory=False)

In [4]:
vehicles = pd.read_csv('Traffic_Crashes_-_Vehicles.csv', low_memory=False)

In [5]:
def howmanyunique(data):
    tempo = []
    nombre = []
    for x in data.columns:
        tempo.append(len(data[x].value_counts().unique()))
        nombre.append(x)
    print(tempo)
    print('')
    print(nombre)

In [6]:
def howmanynan(data):
    print(data.isna().sum())   

In [50]:
def rows_w_nan(data):
    for x in data.columns:
        if data[x].isna().sum() > 0:
            print("Row: {} : {} NaN Values.".format(x, data[x].isna().sum()))
            

In [7]:
def howmanyduplicates(data):
    print(data.duplicated().sum())

# Crashes Data Set

In [8]:
crashes.shape

(482866, 49)

In [9]:
howmanyunique(crashes)

[1, 1, 2, 21, 28, 19, 8, 12, 6, 18, 20, 22, 6, 7, 7, 3, 2, 2, 2, 2, 3, 12, 40, 40, 424, 4, 488, 254, 2, 2, 2, 2, 4, 2, 13, 5, 14, 5, 8, 14, 12, 24, 1, 24, 7, 12, 138, 138, 138]

['CRASH_RECORD_ID', 'RD_NO', 'CRASH_DATE_EST_I', 'CRASH_DATE', 'POSTED_SPEED_LIMIT', 'TRAFFIC_CONTROL_DEVICE', 'DEVICE_CONDITION', 'WEATHER_CONDITION', 'LIGHTING_CONDITION', 'FIRST_CRASH_TYPE', 'TRAFFICWAY_TYPE', 'LANE_CNT', 'ALIGNMENT', 'ROADWAY_SURFACE_COND', 'ROAD_DEFECT', 'REPORT_TYPE', 'CRASH_TYPE', 'INTERSECTION_RELATED_I', 'NOT_RIGHT_OF_WAY_I', 'HIT_AND_RUN_I', 'DAMAGE', 'DATE_POLICE_NOTIFIED', 'PRIM_CONTRIBUTORY_CAUSE', 'SEC_CONTRIBUTORY_CAUSE', 'STREET_NO', 'STREET_DIRECTION', 'STREET_NAME', 'BEAT_OF_OCCURRENCE', 'PHOTOS_TAKEN_I', 'STATEMENTS_TAKEN_I', 'DOORING_I', 'WORK_ZONE_I', 'WORK_ZONE_TYPE', 'WORKERS_PRESENT_I', 'NUM_UNITS', 'MOST_SEVERE_INJURY', 'INJURIES_TOTAL', 'INJURIES_FATAL', 'INJURIES_INCAPACITATING', 'INJURIES_NON_INCAPACITATING', 'INJURIES_REPORTED_NOT_EVIDENT', 'INJURIES_NO_INDICATION',

In [10]:
howmanyduplicates(crashes)

0


In [103]:
rows_w_nan(crashes)

Row: POSTED_SPEED_LIMIT : 10787 NaN Values.


In [12]:
# We are going to drop all the columns with a lot of of NaN values, except for location that we can say right ahead that is
# a multicolinearity case, since it has the Latiutude and Longitude together
crashes = crashes.drop(columns=['CRASH_DATE_EST_I', 'LANE_CNT', 'INTERSECTION_RELATED_I',
                                'NOT_RIGHT_OF_WAY_I', 'HIT_AND_RUN_I', 'PHOTOS_TAKEN_I', 
                                'STATEMENTS_TAKEN_I', 'DOORING_I', 'WORK_ZONE_I', 'WORK_ZONE_TYPE',
                                'WORKERS_PRESENT_I', 'LOCATION'])

### Data Manipulation with Missing Values (NaN)

> **RD_NO**

So we can see here that RD_NO and REPORT_TYPE still have a lot of missing data and that can be explained by the website, where they say and I quote:


> " For privacy reasons, this column is blank for recent crashes."

So since the RD_NO is another way to connect the cases between datasets, we can easily discard the whole column, because we still have CRASH_RECORD_ID, that has no missing values

> **REPORT_TYPE**

In [53]:
crashes.REPORT_TYPE.value_counts()

NOT ON SCENE (DESK REPORT)    277308
ON SCENE                      192629
AMENDED                          240
Name: REPORT_TYPE, dtype: int64

By checking the REPORT_TYPE column we realize that it doesn't offer any predictive value to our model, but still might use it for the business understanding part.

> So we decide to drop the RD_NO columns because we have a good substitute and drop the rows where the remaining NaN values are, so we have a cleaner dataset

In [54]:
crashes = crashes.drop(columns=['RD_NO'])

In [58]:
crashes.dropna(inplace=True)

In [104]:
rows_w_nan(crashes)

Row: POSTED_SPEED_LIMIT : 10787 NaN Values.


### Since "Injuries_Fatal" is our target, let's give some more attention to it

We want to transform the column into a Yes or No kind of answer instead of numbers of deads per accident.

In [15]:
crashes.INJURIES_FATAL.value_counts(dropna=False)

0.0    481428
1.0       437
2.0        28
3.0         5
4.0         1
Name: INJURIES_FATAL, dtype: int64

In [16]:
crashes['INJURIES_FATAL'] = np.where(crashes['INJURIES_FATAL']>0, 1, 0)

In [17]:
crashes.INJURIES_FATAL.value_counts(dropna=False)

0    481428
1       471
Name: INJURIES_FATAL, dtype: int64

## Some more data manipulation with useful columns

### WEATHER_CONDITION

In [97]:
crashes.WEATHER_CONDITION.value_counts()

CLEAR              368594
RAIN                41291
UNKNOWN             20477
SNOW                20185
CLOUDY/OVERCAST     13905
OTHER                3087
Name: WEATHER_CONDITION, dtype: int64

In [92]:
crashes.WEATHER_CONDITION = np.where(crashes.WEATHER_CONDITION=='BLOWING SNOW', 'SNOW', crashes.WEATHER_CONDITION)

In [ ]:
crashes.WEATHER_CONDITION = np.where(crashes.WEATHER_CONDITION=='FREEZING RAIN/DRIZZLE', 'RAIN', crashes.WEATHER_CONDITION)

In [88]:
crashes.WEATHER_CONDITION = np.where(crashes.WEATHER_CONDITION=='FOG/SMOKE/HAZE', 'OTHER', crashes.WEATHER_CONDITION)

In [90]:
crashes.WEATHER_CONDITION = np.where(crashes.WEATHER_CONDITION=='SLEET/HAIL', 'OTHER', crashes.WEATHER_CONDITION)

In [94]:
crashes.WEATHER_CONDITION = np.where(crashes.WEATHER_CONDITION=='BLOWING SAND, SOIL, DIRT', 'OTHER', crashes.WEATHER_CONDITION)

In [96]:
crashes.WEATHER_CONDITION = np.where(crashes.WEATHER_CONDITION=='SEVERE CROSS WIND GATE', 'OTHER', crashes.WEATHER_CONDITION)

In [98]:
crashes.WEATHER_CONDITION.value_counts()

CLEAR              368594
RAIN                41291
UNKNOWN             20477
SNOW                20185
CLOUDY/OVERCAST     13905
OTHER                3087
Name: WEATHER_CONDITION, dtype: int64

### ALIGNMENT

In [66]:
crashes.ALIGNMENT.value_counts()

STRAIGHT AND LEVEL       455951
STRAIGHT ON GRADE          5759
CURVE, LEVEL               3501
STRAIGHT ON HILLCREST      1437
CURVE ON GRADE              664
CURVE ON HILLCREST          227
Name: ALIGNMENT, dtype: int64

### POSTED_SPEED_LIMIT

In [101]:
list(crashes.POSTED_SPEED_LIMIT.value_counts())

[344753, 32178, 28405, 18071, 16305, 9936, 4324, 3714, 2685, 95]

In [102]:
crashes.POSTED_SPEED_LIMIT = pd.cut(crashes.POSTED_SPEED_LIMIT,[0, 5, 10, 15, 20, 25, 30, 35, 40, 45],
                                    precision=0, labels=[0, 1, 2, 3, 4, 5, 6, 7, 8])

In [105]:
crashes.POSTED_SPEED_LIMIT.value_counts()

0    417470
1     39282
8         0
7         0
6         0
5         0
4         0
3         0
2         0
Name: POSTED_SPEED_LIMIT, dtype: int64

In [111]:
howmanyunique(crashes)

[1, 21, 3, 19, 8, 6, 6, 18, 19, 6, 7, 7, 3, 2, 3, 12, 40, 40, 410, 4, 485, 251, 13, 5, 14, 2, 7, 13, 12, 25, 1, 24, 7, 12, 135, 135]

['CRASH_RECORD_ID', 'CRASH_DATE', 'POSTED_SPEED_LIMIT', 'TRAFFIC_CONTROL_DEVICE', 'DEVICE_CONDITION', 'WEATHER_CONDITION', 'LIGHTING_CONDITION', 'FIRST_CRASH_TYPE', 'TRAFFICWAY_TYPE', 'ALIGNMENT', 'ROADWAY_SURFACE_COND', 'ROAD_DEFECT', 'REPORT_TYPE', 'CRASH_TYPE', 'DAMAGE', 'DATE_POLICE_NOTIFIED', 'PRIM_CONTRIBUTORY_CAUSE', 'SEC_CONTRIBUTORY_CAUSE', 'STREET_NO', 'STREET_DIRECTION', 'STREET_NAME', 'BEAT_OF_OCCURRENCE', 'NUM_UNITS', 'MOST_SEVERE_INJURY', 'INJURIES_TOTAL', 'INJURIES_FATAL', 'INJURIES_INCAPACITATING', 'INJURIES_NON_INCAPACITATING', 'INJURIES_REPORTED_NOT_EVIDENT', 'INJURIES_NO_INDICATION', 'INJURIES_UNKNOWN', 'CRASH_HOUR', 'CRASH_DAY_OF_WEEK', 'CRASH_MONTH', 'LATITUDE', 'LONGITUDE']


In [132]:
crashes.DAMAGE.value_counts()

OVER $1,500      269655
$501 - $1,500    137408
$500 OR LESS      60476
Name: DAMAGE, dtype: int64

In [133]:
crashes.CRASH_DAY_OF_WEEK.value_counts()

6    75975
7    68977
3    67341
5    67141
4    66813
2    64997
1    56295
Name: CRASH_DAY_OF_WEEK, dtype: int64

In [134]:
crashes.CRASH_MONTH.value_counts()

10    45053
12    42845
9     41972
11    41620
8     40634
1     40077
2     40014
7     38888
6     36792
5     35784
3     33395
4     30465
Name: CRASH_MONTH, dtype: int64

In [137]:
crashes.CRASH_HOUR.value_counts()

2    196621
1    141033
3     78321
0     42182
Name: CRASH_HOUR, dtype: int64

In [136]:
crashes.CRASH_HOUR = pd.cut(crashes.CRASH_HOUR,[0, 6, 12, 18, 24],
                                    precision=0, labels=[0, 1, 2, 3])

In [138]:
crashes.CRASH_HOUR.value_counts()

2    196621
1    141033
3     78321
0     42182
Name: CRASH_HOUR, dtype: int64

In [140]:
crashes.ROADWAY_SURFACE_COND.value_counts()

DRY                347220
WET                 64424
UNKNOWN             31103
SNOW OR SLUSH       19652
ICE                  3809
OTHER                1121
SAND, MUD, DIRT       210
Name: ROADWAY_SURFACE_COND, dtype: int64

In [144]:
crashes.TRAFFICWAY_TYPE.value_counts()

NOT DIVIDED                        209149
DIVIDED - W/MEDIAN (NOT RAISED)     82824
ONE-WAY                             61863
PARKING LOT                         33176
DIVIDED - W/MEDIAN BARRIER          27706
FOUR WAY                            14558
OTHER                               13135
ALLEY                                7710
UNKNOWN                              5102
CENTER TURN LANE                     3988
T-INTERSECTION                       3092
DRIVEWAY                             1619
RAMP                                 1440
UNKNOWN INTERSECTION TYPE             904
FIVE POINT, OR MORE                   369
Y-INTERSECTION                        369
TRAFFIC ROUTE                         270
NOT REPORTED                          118
ROUNDABOUT                             94
L-INTERSECTION                         53
Name: TRAFFICWAY_TYPE, dtype: int64

In [143]:
# We could check to see how long the street is OR
# We could create a kind of multi-column based on the other column "TRAFFICWAY_TYPE" OR
# Just group them together by that same column
crashes.STREET_NAME.value_counts()

WESTERN AVE      12865
PULASKI RD       11213
CICERO AVE       10233
ASHLAND AVE      10219
HALSTED ST        8955
                 ...  
KINZUA AVE           1
ELSDON AVE           1
SACRAMENTO SD        1
17TH PL              1
STARK ST             1
Name: STREET_NAME, Length: 1520, dtype: int64

In [145]:
crashes.STREET_DIRECTION.value_counts()

W    166524
S    155240
N    113200
E     32575
Name: STREET_DIRECTION, dtype: int64

### **To Delete?**
- 'FIRST_CRASH_TYPE'
- 'INJURIES_INCAPACITATING' 
- 'INJURIES_NON_INCAPACITATING'
- 'INJURIES_REPORTED_NOT_EVIDENT'
- 'INJURIES_TOTAL'
- 'MOST_SEVERE_INJURY'
- 'NUM_UNITS'
- 'BEAT_OF_OCCURRENCE'
- 'DATE_POLICE_NOTIFIED'
- 'PRIM_CONTRIBUTORY_CAUSE'
- 'SEC_CONTRIBUTORY_CAUSE'

# People Data Set

In [18]:
people.shape

(1067653, 30)

In [19]:
howmanyunique(people)

[1, 6, 45, 45, 43, 51, 11, 341, 49, 362, 3, 107, 72, 61, 18, 7, 5, 5, 184, 176, 96, 20, 14, 12, 23, 4, 8, 4, 29, 2]

['PERSON_ID', 'PERSON_TYPE', 'CRASH_RECORD_ID', 'RD_NO', 'VEHICLE_ID', 'CRASH_DATE', 'SEAT_NO', 'CITY', 'STATE', 'ZIPCODE', 'SEX', 'AGE', 'DRIVERS_LICENSE_STATE', 'DRIVERS_LICENSE_CLASS', 'SAFETY_EQUIPMENT', 'AIRBAG_DEPLOYED', 'EJECTION', 'INJURY_CLASSIFICATION', 'HOSPITAL', 'EMS_AGENCY', 'EMS_RUN_NO', 'DRIVER_ACTION', 'DRIVER_VISION', 'PHYSICAL_CONDITION', 'PEDPEDAL_ACTION', 'PEDPEDAL_VISIBILITY', 'PEDPEDAL_LOCATION', 'BAC_RESULT', 'BAC_RESULT VALUE', 'CELL_PHONE_USE']


In [20]:
howmanyduplicates(people)

0


In [149]:
rows_w_nan(people)

Row: RD_NO : 7837 NaN Values.
Row: VEHICLE_ID : 21126 NaN Values.
Row: SEAT_NO : 849503 NaN Values.
Row: CITY : 278316 NaN Values.
Row: STATE : 269523 NaN Values.
Row: ZIPCODE : 346783 NaN Values.
Row: SEX : 15808 NaN Values.
Row: AGE : 304206 NaN Values.
Row: DRIVERS_LICENSE_STATE : 434424 NaN Values.
Row: DRIVERS_LICENSE_CLASS : 517814 NaN Values.
Row: SAFETY_EQUIPMENT : 3169 NaN Values.
Row: AIRBAG_DEPLOYED : 20222 NaN Values.
Row: EJECTION : 13178 NaN Values.
Row: INJURY_CLASSIFICATION : 565 NaN Values.
Row: HOSPITAL : 873244 NaN Values.
Row: EMS_AGENCY : 944181 NaN Values.
Row: EMS_RUN_NO : 1047577 NaN Values.
Row: DRIVER_ACTION : 220203 NaN Values.
Row: DRIVER_VISION : 220482 NaN Values.
Row: PHYSICAL_CONDITION : 219592 NaN Values.
Row: PEDPEDAL_ACTION : 1047794 NaN Values.
Row: PEDPEDAL_VISIBILITY : 1047837 NaN Values.
Row: PEDPEDAL_LOCATION : 1047793 NaN Values.
Row: BAC_RESULT : 219095 NaN Values.
Row: BAC_RESULT VALUE : 1066278 NaN Values.
Row: CELL_PHONE_USE : 1066496 NaN Va

In [147]:
people.INJURY_CLASSIFICATION.value_counts()

NO INDICATION OF INJURY     981284
NONINCAPACITATING INJURY     47541
REPORTED, NOT EVIDENT        28330
INCAPACITATING INJURY         9387
FATAL                          546
Name: INJURY_CLASSIFICATION, dtype: int64

In [153]:
people.AIRBAG_DEPLOYED.value_counts()

DID NOT DEPLOY                            632000
DEPLOYMENT UNKNOWN                        202558
NOT APPLICABLE                            152601
DEPLOYED, FRONT                            31943
DEPLOYED, COMBINATION                      19694
DEPLOYED, SIDE                              8212
DEPLOYED OTHER (KNEE, AIR, BELT, ETC.)       423
Name: AIRBAG_DEPLOYED, dtype: int64

In [154]:
# Useless?
people.EJECTION.value_counts()

NONE                  991272
UNKNOWN                57551
TOTALLY EJECTED         3952
PARTIALLY EJECTED       1034
TRAPPED/EXTRICATED       666
Name: EJECTION, dtype: int64

In [160]:
people.PHYSICAL_CONDITION.value_counts()

NORMAL                          570076
UNKNOWN                         261953
IMPAIRED - ALCOHOL                4142
REMOVED BY EMS                    3304
OTHER                             2349
FATIGUED/ASLEEP                   2251
EMOTIONAL                         1754
ILLNESS/FAINTED                    766
HAD BEEN DRINKING                  677
IMPAIRED - DRUGS                   498
IMPAIRED - ALCOHOL AND DRUGS       184
MEDICATED                          107
Name: PHYSICAL_CONDITION, dtype: int64

In [161]:
people.PERSON_TYPE.value_counts()

DRIVER                 828802
PASSENGER              218150
PEDESTRIAN              12346
BICYCLE                  7205
NON-MOTOR VEHICLE         951
NON-CONTACT VEHICLE       199
Name: PERSON_TYPE, dtype: int64

# Vehicles Data Set

**SEAT_NO**
- Code for seating position of motor vehicle occupant: 1= driver, 2= center front, 3 = front passenger, 4 = second row left, 5 = second row center, 6 = second row right, 7 = enclosed passengers, 8 = exposed passengers, 9= unknown position, 10 = third row left, 11 = third row center, 12 = third row right

In [22]:
vehicles.shape

(986181, 72)

In [23]:
howmanyunique(vehicles)

[1, 17, 17, 40, 16, 9, 22, 1, 2, 150, 522, 51, 68, 17, 21, 25, 8, 28, 2, 2, 25, 2, 133, 121, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 14, 1, 34, 11, 11, 4, 40, 37, 39, 58, 2, 3, 8, 3, 3, 1, 3, 1, 3, 3, 9, 2, 3, 3, 26, 4, 40, 12, 8, 9, 6, 2, 2, 7]

['CRASH_UNIT_ID', 'CRASH_RECORD_ID', 'RD_NO', 'CRASH_DATE', 'UNIT_NO', 'UNIT_TYPE', 'NUM_PASSENGERS', 'VEHICLE_ID', 'CMRC_VEH_I', 'MAKE', 'MODEL', 'LIC_PLATE_STATE', 'VEHICLE_YEAR', 'VEHICLE_DEFECT', 'VEHICLE_TYPE', 'VEHICLE_USE', 'TRAVEL_DIRECTION', 'MANEUVER', 'TOWED_I', 'FIRE_I', 'OCCUPANT_CNT', 'EXCEED_SPEED_LIMIT_I', 'TOWED_BY', 'TOWED_TO', 'AREA_00_I', 'AREA_01_I', 'AREA_02_I', 'AREA_03_I', 'AREA_04_I', 'AREA_05_I', 'AREA_06_I', 'AREA_07_I', 'AREA_08_I', 'AREA_09_I', 'AREA_10_I', 'AREA_11_I', 'AREA_12_I', 'AREA_99_I', 'FIRST_CONTACT_POINT', 'CMV_ID', 'USDOT_NO', 'CCMC_NO', 'ILCC_NO', 'COMMERCIAL_SRC', 'GVWR', 'CARRIER_NAME', 'CARRIER_STATE', 'CARRIER_CITY', 'HAZMAT_PLACARDS_I', 'HAZMAT_NAME', 'UN_NO', 'HAZMAT_PRESENT_I', 'HAZMAT_REPORT

In [24]:
howmanyduplicates(vehicles)

0


In [61]:
rows_w_nan(vehicles)

Row: RD_NO : 7858 NaN Values.
Row: UNIT_TYPE : 1479 NaN Values.
Row: NUM_PASSENGERS : 838583 NaN Values.
Row: VEHICLE_ID : 22726 NaN Values.
Row: CMRC_VEH_I : 967789 NaN Values.
Row: MAKE : 22731 NaN Values.
Row: MODEL : 22873 NaN Values.
Row: LIC_PLATE_STATE : 104563 NaN Values.
Row: VEHICLE_YEAR : 178719 NaN Values.
Row: VEHICLE_DEFECT : 22726 NaN Values.
Row: VEHICLE_TYPE : 22726 NaN Values.
Row: VEHICLE_USE : 22726 NaN Values.
Row: TRAVEL_DIRECTION : 22726 NaN Values.
Row: MANEUVER : 22726 NaN Values.
Row: TOWED_I : 875221 NaN Values.
Row: FIRE_I : 985468 NaN Values.
Row: OCCUPANT_CNT : 22726 NaN Values.
Row: EXCEED_SPEED_LIMIT_I : 983793 NaN Values.
Row: TOWED_BY : 905157 NaN Values.
Row: TOWED_TO : 935405 NaN Values.
Row: AREA_00_I : 947679 NaN Values.
Row: AREA_01_I : 730721 NaN Values.
Row: AREA_02_I : 817483 NaN Values.
Row: AREA_03_I : 892829 NaN Values.
Row: AREA_04_I : 887725 NaN Values.
Row: AREA_05_I : 836437 NaN Values.
Row: AREA_06_I : 838181 NaN Values.
Row: AREA_07_I 

In [164]:
# Notes:
    # new = old[['A', 'C', 'D']]